In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [70]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples,silhouette_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox, skew
from sklearn.decomposition import PCA, KernelPCA

In [71]:
def hacForNumClusters(X,n):
    for cluster_num in range(2,n):
        hac(X,cluster_num)
        
def hac(X,cluster_num):
    hac = AgglomerativeClustering(n_clusters = cluster_num)
    labels = hac.fit_predict(X)
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    print('clus {}: {}, {}'.format(cluster_num, sil, db))
    return labels
        
def cluster(filepath, num_clus, mult=False):
    df = pd.read_csv(filepath)
    df = df.drop(['Bios'], axis=1)
    
    X = df.to_numpy()
    #minmax scaling
    X_minmax = MinMaxScaler().fit_transform(X)
    
    #dimensionality reduction
    pca = PCA(n_components = 0.99)
    X = pca.fit_transform(X_minmax)
    
    if mult:
        hacForNumClusters(X,num_clus)
    else:
        labels = hac(X,num_clus)
        return labels

In [72]:
print('dataset 1')
labels1 = cluster('./TEST1.csv',30,True)


dataset 1
clus 2: 0.12235658968599077, 2.324817108471529
clus 3: 0.12310580948201447, 1.953528913385784
clus 4: 0.14057097283298264, 1.7832065534952526
clus 5: 0.13062037616800862, 1.6911714070371886
clus 6: 0.144702792506051, 1.554390151635028
clus 7: 0.15655025249360324, 1.4375361542116012
clus 8: 0.1735340949176274, 1.3643807760938174
clus 9: 0.177504533701653, 1.2881069287151043
clus 10: 0.17648241744624052, 1.2260451189603734
clus 11: 0.17667364525751203, 1.1755872535404062
clus 12: 0.17524941512017206, 1.018800857726462
clus 13: 0.18310270790846178, 0.9811657609988056
clus 14: 0.18475638852802329, 0.9554377625747816
clus 15: 0.1820600880774312, 0.927854062336475
clus 16: 0.18720701361897127, 0.8595129379261999
clus 17: 0.19334004554133216, 0.8181948486025262
clus 18: 0.18702398101729764, 0.7386998383250655
clus 19: 0.1817981849052378, 0.6670735684725386
clus 20: 0.17705800984855707, 0.6292237128534461
clus 21: 0.17437849054190713, 0.581125323251557
clus 22: 0.1658747367967631, 0.

In [73]:
print('dataset 1')
labels1 = cluster('./TEST1.csv',3)


dataset 1
clus 3: 0.12310580948201447, 1.953528913385784


In [74]:
import os

#os.mkdir(os.getcwd() + '/' + someWord)
def please(filepath, labels, n, someWord):
    df = pd.read_csv(filepath)
    df['cluster #'] = labels
    #os.mkdir(os.getcwd() + '/' + someWord)
    
    
    for cluster in range(n):
        yes = df[df['cluster #'] == cluster]
        filename = '{}/{}.csv'.format(someWord, str(cluster))
        yes.to_csv(filename)
        
    

In [75]:
please('./TEST1.csv',labels1,3,'woohoo1')


clustering the clusters

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def remove_punct(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct
df['Bios'] = df['Bios'].apply(lambda x: remove_punct(x))
tokenizer = RegexpTokenizer(r'\w+')
df['Bios'] = df['Bios'].apply(lambda x: tokenizer.tokenize(x.lower()))
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
df['Bios'] = df['Bios'].apply(lambda x: remove_stopwords(x))
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

df['Bios'] = df['Bios'].apply(lambda x: word_lemmatizer(x))
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

df['Bios'] = df['Bios'].apply(lambda x: word_stemmer(x))
print(df.head())
#TF-IDF vectorizer
tfv = TfidfVectorizer(stop_words = "english",ngram_range = (1,1))
#transform
vec_text = tfv.fit_transform(df.Bios)
#returns a list of words.
words = tfv.get_feature_names()

kmeans = KMeans(n_clusters = 10, n_init = 17, tol = 0.01, max_iter = 200)
#fit the data 
kmeans.fit(vec_text)
#this loop transforms the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

NameError: name 'df' is not defined

In [59]:
def nlp_cluster_for_cluster_spec(dir_spec):
    dirname = '{}/{}'.format(os.getcwd(),dir_spec)
    for filename in os.listdir(dirname):
        labels = nlp_clus('{}/{}'.format(dirname, filename), 10, True)

In [60]:
nlp_cluster_for_cluster_spec('./woohoo1')

0.9929731916162744
clus 10: 0.12360022842993289, 0.4967414371115811
0.9927527424286925
clus 10: 0.1345629286265202, 0.2960262879519318
0.990804191706378


ValueError: Cannot extract more clusters than samples: 10 clusters where given for a tree with 6 leaves.